# Manually

In [2]:
import numpy as np

# f = w * x
# f = 2 * x
# 를 구현한다고 가정. optimal w = 2 = 도달하고자 하는 값

X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

# model prediction
def forward(x) :
    return w * x

# loss (=MSE)
# MSE = 1/N * (w*x - y)**2
def loss(y, y_pred) : 
    return ((y_pred - y)**2).mean()

# gradient
# dJ/dw = 1/N * 2x(w*x - y)
def gradient(x, y, y_pred) :
    return np.dot(2*x, y_pred - y).mean()

print(f"prediction before training: f(5) = {forward(5):.3f}")

# Training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters) :
    # prediction = forward pass
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)

    # gradients
    dw = gradient(X, Y, y_pred)
    
    # update weights
    w -= learning_rate * dw
    
    if epoch % 2 == 0 :
        print(f"epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}")
    
    
print(f"prediction after training: f(5) = {forward(5):.3f}")    

prediction before training: f(5) = 0.000
epoch 1: w = 1.200, loss = 30.00000000
epoch 3: w = 1.872, loss = 0.76800019
epoch 5: w = 1.980, loss = 0.01966083
epoch 7: w = 1.997, loss = 0.00050332
epoch 9: w = 1.999, loss = 0.00001288
epoch 11: w = 2.000, loss = 0.00000033
epoch 13: w = 2.000, loss = 0.00000001
epoch 15: w = 2.000, loss = 0.00000000
epoch 17: w = 2.000, loss = 0.00000000
epoch 19: w = 2.000, loss = 0.00000000
prediction after training: f(5) = 10.000


## Gradients Computation : PyTorch Autograd

In [3]:
# import numpy as np
import torch

# f = w * x
# f = 2 * x
# 를 구현한다고 가정. optimal w = 2 = 도달하고자 하는 값

# X = np.array([1, 2, 3, 4], dtype=np.float32)
# Y = np.array([2, 4, 6, 8], dtype=np.float32)
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# w = 0.0
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# model prediction
def forward(x) :
    return w * x

# loss (=MSE)
# MSE = 1/N * (w*x - y)**2
def loss(y, y_pred) : 
    return ((y_pred - y)**2).mean()

# gradient
# dJ/dw = 1/N * 2x(w*x - y)
# def gradient(x, y, y_pred) :
#     return np.dot(2*x, y_pred - y).mean()

print(f"prediction before training: f(5) = {forward(5):.3f}")

# Training
learning_rate = 0.01
n_iters = 100 # 20 -> 100

for epoch in range(n_iters) :
    # prediction = forward pass
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)

    # gradients
    # dw = gradient(X, Y, y_pred)
    l.backward() # dl/dw => w.grad
    
    # update weights
    with torch.no_grad() :
        # w -= learning_rate * dw
        w -= learning_rate * w.grad
        
    # zero gradients
    w.grad.zero_()
    
    if epoch % 10 == 0 :
        print(f"epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}")
    
# 위에서 manually 방식으로 했던 것과 비교해보면, loss가 완벽히 일치하지 않고, 더 느리게 수렴하는 것을 볼 수 있다.
# 왜냐하면 manually 방식에서는 gradient를 계산할 때, gradient(x, y, y_pred) 함수를 통해 numerical gradient computation으로 계산했기 때문이다. 
# PyTorch에서 .backward()는 정확한 numerical gradient computation을 하진 않기 때문에 convergence가 더 느린 것이다.
# 따라서 epoch을 20이 아니라 조금 더 늘려주면 loss가 0에 수렴하는 것을 볼 수 있다.
print(f"prediction after training: f(5) = {forward(5):.3f}") 


prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
prediction after training: f(5) = 10.000


## Loss Computation : PyTorch Loss
## Parameter updates : PyTorch Optimizer
## Prediction : PyTorch Model

In [10]:
# 1. Design model (input, output size, forward pass)
# 2. Construct loss and optimizer
# 3. Training loop
#     - forward pass: compute prediction
#     - backward pass: gradients
#     - update weights

import torch
import torch.nn as nn

# f = w * x
# f = 2 * x
# 를 구현한다고 가정. optimal w = 2 = 도달하고자 하는 값

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32) # test data 임의 생성. 5라는 값을 넣었을 때, 10이 나와야 한다.
n_samples, n_features = X.shape
print(f"n_samples: {n_samples}, n_features: {n_features}")

input_size = n_features
output_size = 1 # Linear Regression이기 때문에
# w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)
# model = nn.Linear(input_size, output_size)

class LinearRegression(nn.Module) :
    def __init__(self, input_dim, output_dim) :
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)
        
    def forward(self, x) :
        return self.lin(x)
    
model = LinearRegression(input_size, output_size)    

print(f"prediction before training: f(5) = {model(X_test).item():.3f}")

# Training
learning_rate = 0.01
n_iters = 100 
 
# loss
loss = nn.MSELoss()
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(n_iters) :
    # prediction = forward pass
    y_pred = model(X)
    
    # loss
    l = loss(Y, y_pred)

    # gradients
    l.backward() # dl/dw => w.grad
    
    # update weights
    optimizer.step()
        
    # zero gradients
    optimizer.zero_grad()
    
    if epoch % 10 == 0 :
        [w, b] = model.parameters()
        print(f"epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l:.8f}")
    
print(f"prediction after training: f(5) = {model(X_test).item():.3f}") 


n_samples: 4, n_features: 1
prediction before training: f(5) = 3.121
epoch 1: w = 0.944, loss = 15.68601608
epoch 11: w = 1.850, loss = 0.40662462
epoch 21: w = 1.995, loss = 0.01126453
epoch 31: w = 2.018, loss = 0.00099228
epoch 41: w = 2.021, loss = 0.00068571
epoch 51: w = 2.021, loss = 0.00063936
epoch 61: w = 2.020, loss = 0.00060198
epoch 71: w = 2.020, loss = 0.00056694
epoch 81: w = 2.019, loss = 0.00053394
epoch 91: w = 2.019, loss = 0.00050286
prediction after training: f(5) = 10.037
